In [7]:
# Import libraries
import pandas as pd
import pickle
from pathlib import Path


In [8]:


# Convert month end prices to monthly percentage change and drop na values

all_funds_df = all_funds_df.pct_change().dropna()

In [9]:
all_funds_df.tail()

,VTI,FXAIX,FSMDX,FSSNX,FSGGX,FXNAX,FCONX,BIL,VTSAX,VTIAX,VNQ,VBTLX,VFIAX,VVIAX,VTMGX,VEMAX,VIMAX,VSMAX,VIPSX,VSBSX
Date,,,,,,,,,,,,,,,,,,,,
2022-09-01,0.000604,0.003125,-0.004530,-0.011369,-0.013666,-0.005687,0.000000,-0.001311,0.000828,-0.013100,0.001828,-0.006110,0.003169,0.004564,-0.014604,-0.006991,-0.004857,-0.009022,-0.008494,-0.001026
2022-09-02,-0.009915,-0.010649,-0.006068,-0.007077,-0.003260,0.003813,0.001001,0.000109,-0.010029,-0.004056,-0.013952,0.004098,-0.010677,-0.008139,-0.003120,-0.006747,-0.006469,-0.006106,0.006231,0.002053
2022-09-06,-0.004067,-0.004027,-0.003052,-0.009800,-0.006541,-0.009497,0.000000,0.000000,-0.004491,-0.006664,0.009034,-0.009184,-0.004047,-0.003435,-0.006260,-0.008269,-0.003080,-0.006814,-0.008514,-0.001537
2022-09-07,0.018478,0.018380,0.023345,0.022042,0.006584,0.005753,0.000000,0.000109,0.019094,0.006336,0.019523,0.005149,0.018382,0.015128,0.007087,0.005658,0.023115,0.023282,0.003903,0.001026
2022-09-08,0.007919,0.006714,0.010097,0.008363,0.001635,-0.003813,0.000000,0.000219,0.007618,0.002222,0.002751,-0.003074,0.006731,0.007923,0.003909,-0.002369,0.009595,0.010222,-0.006221,-0.001025


In [16]:
""" fidelity_index_focused_models

parameters (from ML model)- investment_amount, investment_date

Transactions dataframe from start date to today with pct change for relevant funds

Investing/Rebalancing function

YEAR 1
year start date - lookup annual contribution amount / investment amount (for ML model, this will be 1, 2, or 3 month of contributions), 
divide by portfolio weights and create row 1

for each row in range of other df 
cumulative return until next 01-01

if date = 01-01

lookup balance at previous day
lookup and add contribution limit 

total divide by asset allocation (FOR ML this happens every month or investment date)

replace row with new amounts

Create a starting code then a loop through the months?

# Set starting values for 2012 based on weight
portfolio_start = 

# Check total = 100
portfolio_sum = portfolio_start.sum()
print(f"Calcalation check: The portfolio starting value is ${portfolio_sum}.")

# Create new dataframe for 2012 return calc using the 2012 start values
portfolio_return_2015 = pd.DataFrame(portfolio_start_2015)
portfolio_return_2015 = portfolio_return_2015.transpose()

# Rename weighted value index to start date 01-01-2015
portfolio_return_2015 = portfolio_return_2015.reset_index(drop=True)
portfolio_return_2015 = portfolio_return_2015.rename(index={0:'2015-01-01'})

# Append the all_etf_df monthly returns
portfolio_return_2015 = portfolio_return_2015.append(all_etf_df.loc['2015-02-01':'2015-12-31']).sort_index()

# Calculate cumulative return and drop na row
portfolio_return_2015 = (1 + portfolio_return_2015.shift(1)).cumprod().dropna(how='all')

# Drop ETFs not included in portfolio for 2015
portfolio_return_2015 = portfolio_return_2015.dropna(axis=1)

# Combine return for portfolio 
portfolio_return_2015 = portfolio_return_2015.sum(axis=1)

# Review the Series
portfolio_return_2015
"""


'\nTransactions dataframe from start date to today with pct change for relevant funds\n\nInvesting/Rebalancing function\n\nYEAR 1\nyear start date - lookup annual contribution amount / investment amount (for ML model, this will be 1, 2, or 3 month of contributions), \ndivide by portfolio weights and create row 1\n\nfor each row in range of other df \ncumulative return until next 01-01\n\nif date = 01-01\n\nlookup balance at previous day\nlookup and add contribution limit \n\ntotal divide by asset allocation (FOR ML this happens every month or investment date)\n\nreplace row with new amounts\n\nCreate a starting code then a loop through the months?\n\n# Set starting values for 2012 based on weight\nportfolio_start = \n\n# Check total = 100\nportfolio_sum = portfolio_start.sum()\nprint(f"Calcalation check: The portfolio starting value is ${portfolio_sum}.")\n\n# Create new dataframe for 2012 return calc using the 2012 start values\nportfolio_return_2015 = pd.DataFrame(portfolio_start_201

In [61]:
# This function sets up the new holding account with a starting balance of the IRA contribution amount for the year
# This function also creates an investment account with a portfolio attached for investment and rebalancing

# Data start date 2012-01-01

portfolio_investment_start_date = '2013-01-01'
ira_contribution_limits.loc[portfolio_investment_start_date]

portfolio_selection = 'fidelity_index_focused_models'

# Read in portfolio data from pkl file

pkl_file = open(Path(f"./Resources/{portfolio_selection}.pkl"), 'rb')
dataframe = pickle.load(pkl_file)
pkl_file.close()

# load portfolio data back into Dataframe
new_account = dataframe

# Read in etf data from pkl file

pkl_file = open(Path(f"./Resources/ira_contribution_limits.pkl"), 'rb')
dataframe = pickle.load(pkl_file)
pkl_file.close()

ira_contribution = dataframe.loc[portfolio_investment_start_date]['ira_limit'][0]

holding_account = ira_contribution

# This amount will be fed in from the ML model
investment_amount = 2000

# Validate investment amount and update holding account balance
if investment_amount <= holding_account:
    # update holding account balance
    holding_account = holding_account - investment_amount
    validated_amount = investment_amount
    
else: 
    validated_amount = holding_account
    holding_account = 0

new_account = round(validated_amount * new_account/100, 2)
new_account


,FXAIX,FSMDX,FSSNX,FSGGX,FXNAX,FCONX,BIL
2013-01-01,700.0,60.0,80.0,360.0,700.0,60.0,40.0


In [2]:
# Toni, we need to save the variables & functions from a jupyter notebook into a file.py so you can "from file import *"

In [62]:
# Maintenance of account
# append day by day performance
# trigger contribution dates to top up by investment_amount and rebalance
